# **Signal To Spike Conversion** - Pre-Processing of the SEEG Signal (2/2)
This notebook presents the **pre-processing stage 2** the SEEG signal goes through before being fed to the SNN. The pre-processing stages are as follows:
1. **Filtering**: The SEEG signal is bandpass filtered to remove noise and artifacts. The bandpass filter is designed using the Butterworth filter and, since we are working with *iEEG*, the signal is filtered in the ripples and FR bands. The co-occurrence of HFOs in both bands is an optimal prediction of post-surgical seizure freedom by defining an optimal "HFO area" or EZ zone.
2. **Signal-to-Spike Conversion**: To interface and communicate with the silicon neurons in the SNN, the SEEG signal must be converted to spikes.

### Check WD (change if necessary) and file loading

In [53]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


## Checkpoint 1

Right now, we have the filtered SEEG signal in both the ripple and FR bands. The next step is to convert the signal to spikes.

## Let's first load the filtered SEEG signal and the corresponding markers
This notebook only converts 1 SEEG channel to spikes. Therefore, the `.npy` file must contain a single channel.

### Declare the `INPUT_FOLDER` and `RESULTS_FOLDER`

In [54]:
from utils.input import BaselineAlgorithm

# CAREFUL WITH THIS FOLDER TO NOT OVERWRITE THE FILES
DATASET_FILENAME =  "filtered_seeg_csl_ch30-59_ch18"        # "seeg_filtered_subset_90-119_segment500_200"  # "filtered_seeg_ch90-119_ch0"     
INPUT_FOLDER = f"input/{DATASET_FILENAME}"   # f"../subset/results/{DATASET_FILENAME}"   

chosen_baseline_alg_suffix = BaselineAlgorithm.Q3   # BaselineAlgorithm.Q3

BASELINE_FILENAME = "seeg_filtered_subset_90-119_segment500_200"
BASELINE_FILE = f"baseline_results/{BASELINE_FILENAME}_thresholds_{chosen_baseline_alg_suffix}.npy"
RESULTS_FOLDER = f"results/{DATASET_FILENAME}"  # 2 names for the same thing?? custom_subset == seeg_filtered_subset?
PLOTS_FOLDER = f"plots/{DATASET_FILENAME}"

# Create the folder if it does not exist
if not os.path.exists(RESULTS_FOLDER):
    os.makedirs(RESULTS_FOLDER)

In [55]:
import numpy as np
import math
from utils.io import preview_np_array
from utils.input import RIPPLE_BAND_FILENAME, FR_BAND_FILENAME, BOTH_BAND_FILENAME

# Load the filtered seeg signal in the ripple band
ripple_seeg_file_name = f"{INPUT_FOLDER}/{RIPPLE_BAND_FILENAME}_band.npy"
ripple_band_seeg = np.load(f"{ripple_seeg_file_name}")

# Remove the extra inner dimension
ripple_band_seeg = np.squeeze(ripple_band_seeg)

preview_np_array(ripple_band_seeg, "Ripple Band SEEG", edge_items=3)

Ripple Band SEEG Shape: (129239,).
Preview: [1.54122064e-04 2.25023101e-03 1.57302684e-02 ... 2.25298186e+00
 2.13484011e+00 1.23953240e+00]


In [56]:
# Load the filtered seeg signal in the fast ripple band
fr_seeg_file_name = f"{INPUT_FOLDER}/{FR_BAND_FILENAME}_band.npy"
fr_band_seeg = np.load(f"{fr_seeg_file_name}")

# Remove the extra inner dimension
fr_band_seeg = np.squeeze(fr_band_seeg)

preview_np_array(fr_band_seeg, "FR Band SEEG", edge_items=3)

FR Band SEEG Shape: (129239,).
Preview: [ 0.00298259  0.02088804  0.03974846 ... -1.50319269 -0.31585847
  1.35409742]


In [57]:
# Load the filtered seeg signal in the fast ripple band
hfo_seeg_file_name = f"{INPUT_FOLDER}/{BOTH_BAND_FILENAME}_band.npy"
hfo_band_seeg = np.load(f"{hfo_seeg_file_name}")

# Remove the extra inner dimension
hfo_band_seeg = np.squeeze(hfo_band_seeg)

preview_np_array(hfo_band_seeg, "HFO Band SEEG", edge_items=3)

HFO Band SEEG Shape: (129239,).
Preview: [ 0.13138129  1.23212452  4.87171836 ... -1.08183782  0.13139587
 -0.00754367]


In [58]:
# Load the annotated events (For Ripples)
ripple_markers_file_name = f"{INPUT_FOLDER}/markers_{RIPPLE_BAND_FILENAME}_band.npy"
ripple_markers = np.load(f"{ripple_markers_file_name}")

# Remove the extra inner dimension
ripple_markers = np.squeeze(ripple_markers)

preview_np_array(ripple_markers, "ripple_markers", edge_items=3)

ripple_markers Shape: (106,).
Preview: [('Ripple',   522.9492, 0.) ('Ripple',  2225.586 , 0.)
 ('Ripple',  2904.2969, 0.) ... ('Ripple', 60941.895 , 0.)
 ('Ripple', 61297.363 , 0.) ('Ripple', 62207.52  , 0.)]


In [59]:
# Load the annotated events (For Fast Ripples)
fr_markers_file_name = f"{INPUT_FOLDER}/markers_{FR_BAND_FILENAME}_band.npy"
fr_markers = np.load(f"{fr_markers_file_name}")

# Remove the extra inner dimension
fr_markers = np.squeeze(fr_markers)

preview_np_array(fr_markers, "fr_markers", edge_items=3)

fr_markers Shape: (64,).
Preview: [('Fast Ripple',   523.9258, 0.) ('Fast Ripple',  2230.957 , 0.)
 ('Fast Ripple',  2918.457 , 0.) ... ('Fast Ripple', 60944.824 , 0.)
 ('Fast Ripple', 61308.105 , 0.) ('Fast Ripple', 62202.637 , 0.)]


In [60]:
# Load the annotated events (For Fast Ripples)
hfo_markers_file_name = f"{INPUT_FOLDER}/markers_{BOTH_BAND_FILENAME}_band.npy"
hfo_markers = np.load(f"{hfo_markers_file_name}")

# Remove the extra inner dimension
hfo_markers = np.squeeze(hfo_markers)

preview_np_array(hfo_markers, "hfo_markers", edge_items=3)

hfo_markers Shape: (170,).
Preview: [('Ripple',   522.9492, 0.) ('Fast Ripple',   523.9258, 0.)
 ('Ripple',  2225.586 , 0.) ... ('Fast Ripple', 61308.105 , 0.)
 ('Fast Ripple', 62202.637 , 0.) ('Ripple', 62207.52  , 0.)]


### Load the signal-to-spike thresholds calculated from the Baseline Detection

In [61]:
# Load the Baseline Thresholds
baseline_thresholds = np.load(BASELINE_FILE)

# preview_np_array(baseline_thresholds, "baseline_thresholds", edge_items=3)

baseline_ripple_thresh = round(baseline_thresholds[0], 4)
baseline_fr_thresh = round(baseline_thresholds[1], 4)
baseline_hfo_thresh = round(baseline_thresholds[2], 4)

print(f"Baseline Ripple Threshold: {baseline_ripple_thresh}")
print(f"Baseline FR Threshold: {baseline_fr_thresh}")
print(f"Baseline HFO Threshold: {baseline_hfo_thresh}")

Baseline Ripple Threshold: 5.6556
Baseline FR Threshold: 1.6041
Baseline HFO Threshold: 5.152


## Define Global Parameters of the Experiment

In [62]:
from utils.input import SAMPLING_RATE, X_STEP

num_samples = ripple_band_seeg.shape[0]    # 2048 * 120 = 245760

input_duration = (num_samples / SAMPLING_RATE) * 1000

print(f"Input Duration: {input_duration} ms")

Input Duration: 63104.98046875 ms


## Signal-to-Spike Conversion
The signal can be converted to spikes in different ways. First, we will try a method where **two spike trains are generated from the filtered signal**:
- **UP Spike Train**: The spikes are generated based on an increase of the signal's amplitude. The spikes are generated when the signal crosses a certain threshold defined by `threshold_up`.
- **DOWN Spike Train**: The spikes are generated based on a decrease of the signal's amplitude. The spikes are generated when the signal crosses a certain threshold defined by `threshold_down`.

The spike trains are generated by comparing the amount of change in the signal since the last time a spike was generated (UP or DOWN). If the positive/negative amplitude change is greater than the defined threshold, the algorithm stores the current timestep in the respective spike train and takes the new amplitude as the reference for the next comparison.

Another important aspect of this algorithm is to model the time that silicon neurons need before they can generate another spike. Both in hardware and software, we call this time `refractory_period`.

### Configurable Parameters for the Signal-to-Spike Conversion
- `threshold_up`: The threshold for the UP spike train.
- `threshold_down`: The threshold for the DOWN spike train.

The accuracy of this algorithm is heavily dependent on the choice of these parameters. To find the optimal values, we can perform a ***baseline detection*** to determine the optimal spike generation threshold automatically for the signal conversion.

**As a first solution, we set these values manually to have a working prototype. Later, we will find the optimal values and compare the results of both methods.**

In [63]:
# Define variables of the Signal to Spike Conversion Manually
ripple_threshold_up = baseline_ripple_thresh  # Threshold for the UP spike detection (in μV)
ripple_threshold_down = -baseline_ripple_thresh # Threshold for the DOWN spike detection (in μV)

fr_threshold_up = baseline_fr_thresh   # Threshold for the UP spike detection (in μV)
fr_threshold_down = -baseline_fr_thresh # Threshold for the DOWN spike detection (in μV)

hfo_threshold_up = baseline_hfo_thresh   # Threshold for the UP spike detection (in μV)
hfo_threshold_down = -baseline_hfo_thresh # Threshold for the DOWN spike detection (in μV)

In [64]:
# Interactive Plot for the HFO detection
# bokeh docs: https://docs.bokeh.org/en/2.4.1/docs/first_steps/first_steps_1.html

from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

# Define the x and y values
# Should the first input start at 0 or X_STEP?
# TODO: is it okay to create a range with floats?
x = [val for val in np.arange(X_STEP, input_duration + X_STEP, X_STEP)] 

In [65]:
from hfo.signal_to_spike.signal_to_spike import signal_to_spike, SignalToSpikeParameters

# Convert the filtered ripple signal to spikes
ripple_spike_trains = signal_to_spike(
    SignalToSpikeParameters(
        signal=ripple_band_seeg, times=np.array(x),
        threshold_up=ripple_threshold_up, threshold_down=ripple_threshold_down,
        # refractory_period=0.002, interpolation_factor=1
        )
)

np.set_printoptions(edgeitems=5)
print("Ripple UP Spike Train shape: ", ripple_spike_trains.up.shape, "Preview: ", ripple_spike_trains.up)
print("Ripple DOWN Spike Train shape: ", ripple_spike_trains.down.shape, "Preview: ", ripple_spike_trains.down)

Ripple UP Spike Train shape:  (7577,) Preview:  [7.81250000e+00 8.30078125e+00 8.78906250e+00 1.26953125e+01
 1.31835938e+01 ... 6.30546875e+04 6.30625000e+04 6.30668945e+04
 6.30732422e+04 6.30747070e+04]
Ripple DOWN Spike Train shape:  (7560,) Preview:  [5.85937500e+00 1.02539062e+01 1.07421875e+01 1.12304688e+01
 1.17187500e+01 ... 6.30463867e+04 6.30590820e+04 6.30605469e+04
 6.30703125e+04 6.30712891e+04]


In [66]:
# Convert the filtered FR signal to spikes
fr_spike_trains = signal_to_spike(
    SignalToSpikeParameters(
        signal=fr_band_seeg, times=np.array(x),
        threshold_up=fr_threshold_up, threshold_down=fr_threshold_down,
        # refractory_period=0.002, interpolation_factor=1   # TODO: Add refractory period
        )
)

np.set_printoptions(edgeitems=5)
print("Fast Ripple UP Spike Train shape: ", fr_spike_trains.up.shape, "Preview: ", fr_spike_trains.up)
print("Fast Ripple DOWN Spike Train shape: ", fr_spike_trains.down.shape, "Preview: ", fr_spike_trains.down)

Fast Ripple UP Spike Train shape:  (21550,) Preview:  [3.90625000e+00 5.85937500e+00 6.34765625e+00 8.30078125e+00
 8.78906250e+00 ... 6.30917969e+04 6.30961914e+04 6.30991211e+04
 6.31020508e+04 6.31049805e+04]
Fast Ripple DOWN Spike Train shape:  (21526,) Preview:  [4.39453125e+00 4.88281250e+00 6.83593750e+00 7.32421875e+00
 7.81250000e+00 ... 6.30903320e+04 6.30932617e+04 6.30971680e+04
 6.31005859e+04 6.31035156e+04]


In [67]:
# Convert the filtered FR signal to spikes
hfo_spike_trains = signal_to_spike(
    SignalToSpikeParameters(
        signal=hfo_band_seeg, times=np.array(x),
        threshold_up=hfo_threshold_up, threshold_down=hfo_threshold_down,
        # refractory_period=0.002, interpolation_factor=1   # TODO: Add refractory period
        )
)

np.set_printoptions(edgeitems=5)
print("HFO UP Spike Train shape: ", hfo_spike_trains.up.shape, "Preview: ", hfo_spike_trains.up)
print("HFO DOWN Spike Train shape: ", hfo_spike_trains.down.shape, "Preview: ", hfo_spike_trains.down)

HFO UP Spike Train shape:  (10577,) Preview:  [1.95312500e+00 4.39453125e+00 4.88281250e+00 5.37109375e+00
 1.12304688e+01 ... 6.30532227e+04 6.30644531e+04 6.30654297e+04
 6.30786133e+04 6.30922852e+04]
HFO DOWN Spike Train shape:  (10594,) Preview:  [2.92968750e+00 3.41796875e+00 5.85937500e+00 6.34765625e+00
 7.32421875e+00 ... 6.30571289e+04 6.30673828e+04 6.30683594e+04
 6.30893555e+04 6.31025391e+04]


## Visualize the Spike Trains
Let's plot the generated spike trains via a raster plot.

In [68]:
from utils.raster_plot import create_raster_fig

# ------------------------------------------------------------------------------- #
# ------- Create the raster plot for the Ripple UP and DOWN spike trains -------- #
# ------------------------------------------------------------------------------- #

# Create a list containing the x values for the raster plot.
ripple_raster_x = np.concatenate((ripple_spike_trains.up, ripple_spike_trains.down), axis=0)

# Create a list containing the y values for the raster plot.
# The UP spike train will be represented by 1s and the DOWN spike train by 0s
ripple_raster_y = [1 for _ in range(len(ripple_spike_trains.up))] + [0 for _ in range(len(ripple_spike_trains.down))]

ripple_train_raster = create_raster_fig("Ripple UP and DOWN spike events", "Time (ms)", "Channel", ripple_raster_x, 
                                        ripple_raster_y, y_axis_ticker=[0, 1])

In [69]:
import bokeh.plotting as bplt
showRasterPlot = True

# Plot the raster plot for the Ripple spike trains
if showRasterPlot:
    bplt.show(ripple_train_raster)

In [70]:
# ------------------------------------------------------------------------------- #
# ------- Create the raster plot for the Fast Ripple UP and DOWN spike trains -------- #
# ------------------------------------------------------------------------------- #

# Create a list containing the x values for the raster plot.
fr_raster_x = np.concatenate((fr_spike_trains.up, fr_spike_trains.down), axis=0)

# Create a list containing the y values for the raster plot.
# The UP spike train will be represented by 1s and the DOWN spike train by 0s
fr_raster_y = [1 for _ in range(len(fr_spike_trains.up))] + [0 for _ in range(len(fr_spike_trains.down))]

fr_train_raster = create_raster_fig("Fast Ripple UP and DOWN spike events", "Time (ms)", "Channel", fr_raster_x,
                                    fr_raster_y, y_axis_ticker=[0, 1])

In [71]:
# Plot the raster plot for the Fast Ripple spike trains
if showRasterPlot:
    bplt.show(fr_train_raster)

In [72]:
# ------------------------------------------------------------------------------- #
# ------- Create the raster plot for the HFO UP and DOWN spike trains -------- #
# ------------------------------------------------------------------------------- #

# Create a list containing the x values for the raster plot.
hfo_raster_x = np.concatenate((hfo_spike_trains.up, hfo_spike_trains.down), axis=0)

# Create a list containing the y values for the raster plot.
# The UP spike train will be represented by 1s and the DOWN spike train by 0s
hfo_raster_y = [1 for _ in range(len(hfo_spike_trains.up))] + [0 for _ in range(len(hfo_spike_trains.down))]

hfo_train_raster = create_raster_fig("HFO UP and DOWN spike events", "Time (ms)", "Channel", hfo_raster_x,
                                    hfo_raster_y, y_axis_ticker=[0, 1])

In [73]:
# Plot the raster plot for the Fast Ripple spike trains
if showRasterPlot:
    bplt.show(hfo_train_raster)

### Export the Raster Plots 

In [74]:
EXPORT_RIPPLE_RASTER = True
if EXPORT_RIPPLE_RASTER:
    ripple_raster_filename = f"{PLOTS_FOLDER}/ripple_train_raster_thresholds{ripple_threshold_up}-{ripple_threshold_down}.html"

    # Create folder if it does not exist
    if not os.path.exists(PLOTS_FOLDER):
        os.makedirs(PLOTS_FOLDER)

    # Customize the output file settings
    bplt.output_file(filename=ripple_raster_filename, title="Ripple Spike Train Raster Plot")

    # Save the plot
    bplt.save(ripple_train_raster)

In [75]:
EXPORT_FR_RASTER = True
if EXPORT_FR_RASTER:
    fr_raster_filename = f"{PLOTS_FOLDER}/fr_train_raster_thresholds{fr_threshold_up}-{fr_threshold_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=fr_raster_filename, title="Fast Ripple Spike Train Raster Plot")

    # Save the plot
    bplt.save(fr_train_raster)

In [76]:
EXPORT_HFO_RASTER = True
if EXPORT_HFO_RASTER:
    hfo_raster_filename = f"{PLOTS_FOLDER}/hfo_train_raster_thresholds{hfo_threshold_up}-{hfo_threshold_down}.html"

    # Customize the output file settings
    bplt.output_file(filename=hfo_raster_filename, title="HFO Spike Train Raster Plot")

    # Save the plot
    bplt.save(hfo_train_raster)

## Export the Spike Trains to CSV Files for the Lava SNN

We have successfully converted the SEEG signal to spikes. The next step is to feed these spikes to the SNN for Ripple and Fast Ripple detection.

For this, we will create a file for each type of spike train (UP and DOWN). I'm not sure if we should join the spikes of both bands in a single file or keep them separate.

In [77]:
# Create a csv file with the spike train data
import csv

def write_spike_train_to_csv(file_name, spike_train, channel_idx):
    """
    This function writes the spike train to a csv file.
    @file_name (str): Name of the file to be created.
    @spike_train (np.ndarray): Array with the spike train data.
    @channel_idx (int): Index of the channel that generated the spike train. (According to the original data)
    """
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["time", "channel_idx"])
        for spike_time in spike_train:
            writer.writerow([spike_time, channel_idx])

In [78]:
WRITE_RIPPLE_CSV_FILES = True
WRITE_FR_CSV_FILES = True
WRITE_HFO_CSV_FILES = True

In [79]:
selected_ch_idx = -1    # TODO: Should we really send the channel?

if WRITE_RIPPLE_CSV_FILES:
    # Create the csv file for the Ripple UP spike train
    ripple_up_file_name = f"{RESULTS_FOLDER}/ripple_up_spike_train_{ripple_threshold_up}.csv"
    write_spike_train_to_csv(ripple_up_file_name, ripple_spike_trains.up, selected_ch_idx)

    # Create the csv file for the Ripple DOWN spike train
    ripple_down_file_name = f"{RESULTS_FOLDER}/ripple_down_spike_train_{ripple_threshold_down}.csv"
    write_spike_train_to_csv(ripple_down_file_name, ripple_spike_trains.down, selected_ch_idx)

In [80]:
if WRITE_FR_CSV_FILES:
    # Create the csv file for the Fast Ripple UP spike train
    fr_up_file_name = f"{RESULTS_FOLDER}/fr_up_spike_train_{fr_threshold_up}.csv"
    write_spike_train_to_csv(fr_up_file_name, fr_spike_trains.up, selected_ch_idx)

    # Create the csv file for the Fast Ripple DOWN spike train
    fr_down_file_name = f"{RESULTS_FOLDER}/fr_down_spike_train_{fr_threshold_down}.csv"
    write_spike_train_to_csv(fr_down_file_name, fr_spike_trains.down, selected_ch_idx)

In [81]:
if WRITE_HFO_CSV_FILES:
    # Create the csv file for the Fast Ripple UP spike train
    hfo_up_file_name = f"{RESULTS_FOLDER}/hfo_up_spike_train_{hfo_threshold_up}.csv"
    write_spike_train_to_csv(hfo_up_file_name, hfo_spike_trains.up, selected_ch_idx)

    # Create the csv file for the Fast Ripple DOWN spike train
    hfo_down_file_name = f"{RESULTS_FOLDER}/hfo_down_spike_train_{hfo_threshold_down}.csv"
    write_spike_train_to_csv(hfo_down_file_name, hfo_spike_trains.down, selected_ch_idx)

## Generate the input files for the SNN

### Is the SNN going to detect the HFO events in windows? Or do we take it as a continous input?
**If so**: The SNN is going to detect HFO events in windows. Therefore, the input to the network must be organized in windows of a certain size. The size of the window is a hyperparameter that can be tuned to improve the performance of the network.

I think windowing makes more sense when we are learning with an ANN. Since we want real-time detection, feeding a continous input makes more sense.

### Let's assume we do NOT need to window the input
In this case, our input will simply be a continous stream of spikes. We can feed the spikes to the SNN in real-time. At each timestep, the SNN will receive 2 binary inputs (UP and DOWN spikes) indicating the presence of a spike in the respective spike train.

In [82]:
# Create a numpy array that will store the input (2D)
snn_input = np.zeros((num_samples, 2))  # 2 columns: UP and DOWN spike trains

# ---------------------------------------------------------------------------------- #
# -------- Select the Spike Trains to be used as input for the SNN ----------------- #
# ---------------------------------------------------------------------------------- #
selected_up_spikes = ripple_spike_trains.up
selected_down_spikes = ripple_spike_trains.down

# Iterate the time steps of the recording and check if there are spikes in the selected spike trains at each timestep
curr_up_idx = 0
curr_down_idx = 0
for (idx, time_step) in enumerate(x):
    # Check if an UP spike occurs at this time step
    if curr_up_idx < len(selected_up_spikes) and selected_up_spikes[curr_up_idx] <= time_step:
        snn_input[idx][0] = 1   # Mark the UP spike in the input array
        curr_up_idx += 1    # Move to the next spike in the UP spike train
    # Check if a DOWN spike occurs at this time step
    elif curr_down_idx < len(selected_down_spikes) and selected_down_spikes[curr_down_idx] <= time_step:
        snn_input[idx][1] = 1   # Mark the DOWN spike in the input array
        curr_down_idx += 1  # Move to the next spike in the DOWN spike train
    
    if curr_up_idx >= len(selected_up_spikes) and curr_down_idx >= len(selected_down_spikes):
        # All the spikes have been added to the input array
        break

np.set_printoptions(edgeitems=5)
print("snn_input: ", snn_input.shape, "Preview:", snn_input)

snn_input:  (129239, 2) Preview: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [83]:
# ----------------------- DEPRECATED ??? ---------------------- 

# Export the input array to a numpy file
EXPORT_INPUT = False
if EXPORT_INPUT:
    input_file_name = f"{RESULTS_FOLDER}/snn_input_ripple_{ripple_threshold_up}_{ripple_threshold_down}.npy"
    np.save(input_file_name, snn_input)

    EXPORT_INPUT_CSV = True
    if EXPORT_INPUT_CSV:
        # Export to CSV for visualization purposes
        with open(f"{RESULTS_FOLDER}/snn_input_ripple_{ripple_threshold_up}_{ripple_threshold_down}.csv", mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["time", "up_spike", "down_spike"])
            for (idx, time_step) in enumerate(x):
                writer.writerow([time_step, snn_input[idx][0], snn_input[idx][1]])

## Generate the Ground Truth File for the SNN (Target)

Extract the markers from the selected channel

### Create a target file for each band

In [84]:
# The target output for the SNN must have the same length as the input.
ripple_target_np = np.zeros((num_samples))  # 1 column: 0/1 for the output classes (No Event, Ripple/Fast Ripple or both)
# TODO: Could have more than 2 classes to differentiate the labels

# Get the markers for the selected channel
# Each marker has the following keys: label, position, and duration
ripple_selected_ch_markers = ripple_markers
preview_np_array(ripple_selected_ch_markers, "Ripple Selected Channel Markers")

Ripple Selected Channel Markers Shape: (106,).
Preview: [('Ripple',   522.9492, 0.) ('Ripple',  2225.586 , 0.)
 ('Ripple',  2904.2969, 0.) ('Ripple',  4879.3945, 0.)
 ('Ripple',  5058.1055, 0.) ... ('Ripple', 60533.69  , 0.)
 ('Ripple', 60738.77  , 0.) ('Ripple', 60941.895 , 0.)
 ('Ripple', 61297.363 , 0.) ('Ripple', 62207.52  , 0.)]


In [85]:
# The target output for the SNN must have the same length as the input.
fr_target_np = np.zeros((num_samples))  # 1 column: 0/1 for the output classes (No Event, Ripple/Fast Ripple or both)

fr_selected_ch_markers = fr_markers
preview_np_array(fr_selected_ch_markers, "FR Selected Channel Markers")

FR Selected Channel Markers Shape: (64,).
Preview: [('Fast Ripple',   523.9258, 0.) ('Fast Ripple',  2230.957 , 0.)
 ('Fast Ripple',  2918.457 , 0.) ('Fast Ripple',  3234.8633, 0.)
 ('Fast Ripple',  3724.121 , 0.) ... ('Fast Ripple', 59329.59  , 0.)
 ('Fast Ripple', 60741.7   , 0.) ('Fast Ripple', 60944.824 , 0.)
 ('Fast Ripple', 61308.105 , 0.) ('Fast Ripple', 62202.637 , 0.)]


In [86]:
# The target output for the SNN must have the same length as the input.
hfo_target_np = np.zeros((num_samples))  # 1 column: 0/1 for the output classes (No Event, Ripple/Fast Ripple or both)

hfo_selected_ch_markers = hfo_markers
preview_np_array(hfo_selected_ch_markers, "FR Selected Channel Markers")

FR Selected Channel Markers Shape: (170,).
Preview: [('Ripple',   522.9492, 0.) ('Fast Ripple',   523.9258, 0.)
 ('Ripple',  2225.586 , 0.) ('Fast Ripple',  2230.957 , 0.)
 ('Ripple',  2904.2969, 0.) ... ('Fast Ripple', 60944.824 , 0.)
 ('Ripple', 61297.363 , 0.) ('Fast Ripple', 61308.105 , 0.)
 ('Fast Ripple', 62202.637 , 0.) ('Ripple', 62207.52  , 0.)]


### Convert to the format that the learnable SNN (Slayer) can read ----> **This step should be done by the Learnable SNN when it is required**

#### Fill the target numpy array with 1s where the HFOs are present

In [87]:
# ------------ DEPRECATED ----------------

""" from utils.input import label_has_hfo_event

# Iterate the time steps of the recording and check if there is an annotated event at each timestep (Ripple Band)
curr_markers_idx = 0
for (idx, time_step) in enumerate(x):
    # Check if an event occurs at this time step
    if curr_markers_idx < len(ripple_selected_ch_markers) and ripple_selected_ch_markers[curr_markers_idx]['position'] <= time_step:
        # If the label has an HFO event, mark it as 1 in the target array 
        # Can use this function since the given labels are already filtered in the desired band
        if label_has_hfo_event(ripple_selected_ch_markers[curr_markers_idx]['label']):
            ripple_target_np[idx] = 1   # Mark the Labelled event in the target array
        
        curr_markers_idx += 1    # Move to the next annotated event
    
    if curr_markers_idx >= len(ripple_selected_ch_markers):
        # All the spikes have been added to the input array
        break

# Iterate the time steps of the recording and check if there is an annotated event at each timestep (Fast Ripple Band)
curr_markers_idx = 0
for (idx, time_step) in enumerate(x):
    # Check if an event occurs at this time step
    if curr_markers_idx < len(fr_selected_ch_markers) and fr_selected_ch_markers[curr_markers_idx]['position'] <= time_step:
        # If the label has an HFO event, mark it as 1 in the target array 
        # Can use this function since the given labels are already filtered in the desired band
        if label_has_hfo_event(fr_selected_ch_markers[curr_markers_idx]['label']):
            fr_target_np[idx] = 1   # Mark the Labelled event in the target array
        
        curr_markers_idx += 1    # Move to the next annotated event
    
    if curr_markers_idx >= len(fr_selected_ch_markers):
        # All the spikes have been added to the input array
        break """

" from utils.input import label_has_hfo_event\n\n# Iterate the time steps of the recording and check if there is an annotated event at each timestep (Ripple Band)\ncurr_markers_idx = 0\nfor (idx, time_step) in enumerate(x):\n    # Check if an event occurs at this time step\n    if curr_markers_idx < len(ripple_selected_ch_markers) and ripple_selected_ch_markers[curr_markers_idx]['position'] <= time_step:\n        # If the label has an HFO event, mark it as 1 in the target array \n        # Can use this function since the given labels are already filtered in the desired band\n        if label_has_hfo_event(ripple_selected_ch_markers[curr_markers_idx]['label']):\n            ripple_target_np[idx] = 1   # Mark the Labelled event in the target array\n        \n        curr_markers_idx += 1    # Move to the next annotated event\n    \n    if curr_markers_idx >= len(ripple_selected_ch_markers):\n        # All the spikes have been added to the input array\n        break\n\n# Iterate the time 

In [88]:
""" preview_np_array(ripple_target_np, "ripple_target_np")
print(np.count_nonzero(ripple_target_np)) """

' preview_np_array(ripple_target_np, "ripple_target_np")\nprint(np.count_nonzero(ripple_target_np)) '

In [89]:
""" preview_np_array(fr_target_np, "fr_target_np")
print(np.count_nonzero(fr_target_np)) """

' preview_np_array(fr_target_np, "fr_target_np")\nprint(np.count_nonzero(fr_target_np)) '

## Export the target files to numpy files

In [90]:
# Export the target array to a numpy file
EXPORT_TARGET = True
if EXPORT_TARGET:
    ripple_target_file_name = f"{RESULTS_FOLDER}/{RIPPLE_BAND_FILENAME}_ground_truth.npy"
    # Save the ripple target array to a numpy file
    np.save(ripple_target_file_name, ripple_selected_ch_markers)

    # Save the fast ripple target array to a numpy file
    fr_target_file_name = f"{RESULTS_FOLDER}/{FR_BAND_FILENAME}_ground_truth.npy"
    np.save(fr_target_file_name, fr_selected_ch_markers)

    # Save the HFO target array to a numpy file
    hfo_target_file_name = f"{RESULTS_FOLDER}/{BOTH_BAND_FILENAME}_ground_truth.npy"
    np.save(hfo_target_file_name, hfo_selected_ch_markers)